<a href="https://colab.research.google.com/github/LeeSeungYun1020/HandsOnLargeLanguageModels/blob/main/handsOnLLM01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 첫번째 텍스트 생성하기

핸즈온 LLM 1장

- Nvidia cuda 사용을 위해 런타임 교체 필요
- 생각보다 시간 많이 필요...

1. 언어 AI?
  - 인공지능: 인간 지능에 가까운 작업을 수행하는 컴퓨터 시스템
  - 언어 AI: 인간 언어를 이해, 처리, 생성하는 기술에 초점을 맞춘 AI

2. 언어 AI의 역사
  - Bag of Words
    - 토큰화한 단어를 벡터로 표현, text 본질의 의미는 무시됨, 띄어쓰기 없는 언어 적용 불가
  - word2vec
    - RNN 인코더 -> 문맥 임베딩 -> RNN 디코더
    - 임베딩으로 신경망을 학습(구성)하여 유사(경향)성을 측정, 정적이라 문맥 파악 어려움
  - RNN + attention
    - RNN 인코더 -> 어텐션 디코더
    - RNN(순환신경망): 인코딩, 디코딩을 순차, 자기회귀적으로 수행하여 긴 문장을 처리
    - 어텐션: 관련 있는 단어에 가중치를 부여하여 중요도를 판단
  - Transforme(attention only)
    - 트랜스포머 인코더(셀프 어텐션, 피드포워드 신경망) -> 트랜스포더 디코더(마스크드 셀프 어텐션, 인코더 어텐션, 피드포워드 신경망)
    - RNN을 사용하지 않고 어텐션만 사용, 병렬 훈련으로 훈련 속도 향상
    - 셀프 어텐션: 한 시퀀스 안에서 중요도를 판단하여 전체 시퀀스를 한 번에 처리
    - 피드포워드 신경망: 입력이 출력으로 한 방향으로 전달되는 구조의 신경망
    - 마스크드 셀프 어텐션: 미래 위치를 숨겨 앞서 생성된 위치를 중심으로 다음 생성할 단어 고려
  - BERT
    - 언어 표현 목적의 인코더 기반 모델, 전이 학습에 사용
    - 마스크드 언어 모델링 기법 적용: 입력의 특정 부분을 숨긴 뒤 표현(예측)하도록 학습
    - 모델 사전 훈련 후 특정 작업을 위한 미세 튜닝 진행(Ex: 분류, 유사 문장 판별, 개체 인식)
  - GPT
    - 언어 생성 목적의 디코더 기반 모델, 챗봇 훈련에 사용
    - 파라미터를 통해 모델의 언어 이해
    - 문맥 길이가 모델이 처리할 수 있는 최대 토큰 수를 나타내며, 자기회귀 성질에 따라 새로운 토큰이 생성되면서 현재 문맥 길이가 늘어남
    - 표현 모델과 마찬가지로 미세 튜닝 가능(Ex: 채팅)

3. LLM 정의
  - 대규모 언어 모델(LLM)은 일반적으로 디코더 기반 생성 모델을 지칭
  - 아주 임의적인 명칭으로 소규모 언어 모델이 더 강력한 성능을 내거나 생성 기능이 없는 모델이 있는 등 광범위한 범위로 확장

4. 훈련 패러다임
  - 언어 모델링
    - 사전 훈련 단계로 LLM을 훈련시켜 모델이 문법, 맥락, 패턴 학습하여 베이스 모델 생성
    - 많은 계산, 긴 훈련 시간 필요
    - 다음 단어 예측 외 특정 작업에 맞춰지지 않아, 일반적으로 명령에 따르지 못함
  - 미세 튜닝
    - 사후 훈련 단계로 구체적인 작업을 수행하도록 훈련
    - 언어 모델링 단계에 비해 초저비용

5. 앱(용례)
  - 리뷰 긍정도 분석
  - 인기 이슈 파악
  - 관련 문서 검색
  - 외부 자원(문서, 도구)을 활용하는 LLM 챗봇
  - 냉장고 사진으로 레시피 생성

 6. 책임
 - 편향과 공정, 책임과 투명성, 유해 콘텐츠 생성, 지적 재산권, 규제

 7. 인터페이스
  - 비공개(독점) 모델
    - 가중치, 구조 비공개
    - GPT-4(오픈AI), Claude(엔트로픽)
    - 유료 지원 및 서비스, 컴퓨팅 자원 불필요
    - 미세 튜닝 불가, 공급자에 데이터 공유 필요
  - 공개 모델
    - 가중치, 구조 공개
    - Phi(마이크로소프트), Llama(메타), Mistral(코히어)
    - 사용자가 모델 완전히 제어 가능
    - 구체적인 지식과 컴퓨팅 리소스 필요
  - 프레임워크
    - 과다하게 많은 프레임워크 출시 - 사용법이 유사하여 하나를 쓰면 다른걸 쓰는건 쉬움

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [9]:
from transformers import pipelines

In [17]:
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    return_full_text = False,
    max_new_tokens = 500,
    do_sample = False,
    use_cache=False
)


Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [18]:
prompt = "Create a funny joke about chickens"
output = generator(prompt)
print(output[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


.

Chickens are known for their clucking and pecking, but did you know they also have a talent for comedy? Here's a joke about chickens that will make you laugh:

Why did the chicken join the circus?

Because it wanted to be a "cluck-per"!

This joke plays on the word "performance" and the fact that chickens are often associated with clucking. It's a light-hearted way to poke fun at the idea of chickens being entertainers.

Remember, humor is subjective, so feel free to come up with your own chicken-themed jokes!
